In [1]:
import os
import cv2
import json
import random
import uuid

In [2]:
images_path = os.path.join(os.getcwd(), 'segmentation_test', 'images')
labels_path = os.path.join(os.getcwd(), 'segmentation_test', 'labels')

In [3]:
def scale_boxes(file_name, x_scale, y_scale):
    with open(os.path.join(labels_path, file_name + '.json'), 'r') as f:
        data = json.load(f)

    boxes = []
    for points in data['shapes']:
        xmin = int(points['points'][0][0] * x_scale)
        ymin = int(points['points'][0][1] * y_scale)
        xmax = int(points['points'][1][0] * x_scale)
        ymax = int(points['points'][1][1] * y_scale)
        boxes.append([xmin, ymin, xmax, ymax])
    
    return boxes

In [4]:
def get_image_and_scale(file_name, new_size):
    image = cv2.imread(os.path.join(images_path, file_name + '.jpg'))
    y_ = image.shape[0]
    x_ = image.shape[1]

    image = cv2.resize(image, new_size)

    x_scale = new_size[0] / x_
    y_scale = new_size[1] / y_

    boxes = scale_boxes(file_name, x_scale, y_scale)

    return image, boxes

In [5]:
def update_json(file_name):
    with open(os.path.join(labels_path, file_name + '.json'), 'r') as f:
        data = json.load(f)

    new_data = data.copy()

    image, boxes = get_image_and_scale(file_name, (650, 650))

    temp_dict = {'shapes': []}

    for box in boxes:
        box_format = []
        box_format.append([
            box[0], box[1]
        ])
        box_format.append([
            box[2], box[3]
        ])

        temp_dict['shapes'].append(
            {
                'label': 'dieback_ash',
                'points': box_format,
                'group_id': None,
                'description': '',
                'shape_type': 'rectangle',
                'flags': {},
                'mask': None
            }
        )

    new_data['shapes'] = temp_dict['shapes']
    new_uuid = str(uuid.uuid4())
    new_data['imagePath'] = new_uuid + '.jpg'
    new_data['imageHeight'] = 650
    new_data['imageWidth'] = 650
    new_data['imageData'] = None

    return image, new_data

In [6]:
def save_new_data_and_image(file_name):
    base_path = os.path.join(os.getcwd(), 'segmentation_test')
    new_image_path = os.path.join(base_path, 'segmented_image')
    new_label_path = os.path.join(base_path, 'segmented_label')

    image, data = update_json(file_name)
    new_file_name = data['imagePath']
    cv2.imwrite(os.path.join(new_image_path, new_file_name), image)
    new_file_name = new_file_name[:-4] + '.json'
    with open(os.path.join(new_label_path, new_file_name), 'x') as f:
        json.dump(data, f, indent=4)



In [8]:
all_items = os.listdir(labels_path)

for file in all_items:
    file = file[:-5]
    save_new_data_and_image(file)


In [48]:
test_images_path = os.path.join(os.getcwd(), 'segmentation_test', 'segmented_image')
test_labels_path = os.path.join(os.getcwd(), 'segmentation_test', 'segmented_label')

img = cv2.imread(os.path.join(test_images_path, os.listdir(test_images_path)[3]))

with open(os.path.join(test_labels_path, os.listdir(test_labels_path)[3]), 'r') as f:
    data = json.load(f)

for points in data['shapes']:
    xmin = int(points['points'][0][0])
    ymin = int(points['points'][0][1])
    xmax = int(points['points'][1][0])
    ymax = int(points['points'][1][1])
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255,0,0), 2)

cv2.imshow('check', img)

cv2.waitKey(0)

-1